# Base Class Circuit:

In [65]:
import cmath
import math

class Circuit:
    def __init__(self, E):
        self._E = cmath.rect(E[0], math.radians(E[1]))
        self._Z = None  
        self._I = None  

    @property
    def current(self):
        if self._I is None:
            self._I = self._E / self.impedance
        return self._I

    @property
    def power_factor(self):
        angle = cmath.phase(self.impedance)
        return math.cos(angle)

    @staticmethod
    def format_phasor(value):
        magnitude = abs(value)
        angle = math.degrees(cmath.phase(value))
        return f"{magnitude:.2f}∠{angle:.2f}°"

# Subclass R: 

In [68]:
class R(Circuit):
    def __init__(self, E, R1):
        super().__init__(E)
        self._R1 = R1

    @property
    def impedance(self):
        if self._Z is None:
            self._Z = self._R1
        return self._Z

    @property
    def average_power(self):
        I = self.current
        P = abs(I)**2 * self._R1
        return P

# Subclass RCSeries: 

In [71]:
class RCSeries(R):
    def __init__(self, E, R1, XC):
        super().__init__(E, R1)
        self._XC = XC

    @property
    def impedance(self):
        if self._Z is None:
            self._Z = self._R1 - 1j * self._XC
        return self._Z

    @property
    def voltage_drops(self):
        I = self.current
        VR = I * self._R1
        VC = I * (-1j) * self._XC
        return VR, VC


# Subclass RLSeries:

In [74]:
class RLSeries(R):
    def __init__(self, E, R1, XL):
        super().__init__(E, R1)
        self._XL = XL

    @property
    def impedance(self):
        if self._Z is None:
            self._Z = self._R1 + 1j * self._XL
        return self._Z

    @property
    def voltage_drops(self):
        I = self.current
        VR = I * self._R1
        VL = I * 1j * self._XL
        return VR, VL


# Subclass RLCSeries: 

In [77]:
class RLCSeries(R):
    def __init__(self, E, R1, XL, XC):
        super().__init__(E, R1)
        self._XL = XL
        self._XC = XC

    @property
    def impedance(self):
        if self._Z is None:
            self._Z = self._R1 + 1j * (self._XL - self._XC)
        return self._Z

    @property
    def voltage_drops(self):
        I = self.current
        VR = I * self._R1
        VL = I * 1j * self._XL
        VC = I * (-1j) * self._XC
        return VR, VL, VC


# Subclass RCParallel:

In [80]:
class RCParallel(Circuit):
    def __init__(self, E, R1, XC):
        super().__init__(E)
        self._R1 = R1
        self._XC = XC

    @property
    def impedance(self):
        if self._Z is None:
            Y_R = 1 / self._R1
            Y_C = 1j / self._XC
            Yt = Y_R + Y_C
            self._Z = 1 / Yt
        return self._Z

# Subclass RLParallel: 

In [83]:
class RLParallel(Circuit):
    def __init__(self, E, R1, XL):
        super().__init__(E)
        self._R1 = R1
        self._XL = XL

    @property
    def impedance(self):
        if self._Z is None:
            Y_R = 1 / self._R1
            Y_L = -1j / self._XL
            Yt = Y_R + Y_L
            self._Z = 1 / Yt
        return self._Z

# Subclass RLCParallel:

In [86]:
class RLCParallel(Circuit):
    def __init__(self, E, R1, XL, XC):
        super().__init__(E)
        self._R1 = R1
        self._XL = XL
        self._XC = XC

    @property
    def impedance(self):
        if self._Z is None:
            Y_R = 1 / self._R1
            Y_L = -1j / self._XL
            Y_C = 1j / self._XC
            Yt = Y_R + Y_L + Y_C
            self._Z = 1 / Yt
        return self._Z

# # Creating objects for each class with example parameters

In [89]:
# Example voltage
E = (20, 0)
r_circuit = R(E, 10)  
rc_series_circuit = RCSeries(E, 10, 26.53)  
rl_series_circuit = RLSeries(E, 10, 37.70)  
rlc_series_circuit = RLCSeries(E, 10, 37.70, 26.53)  
rc_parallel_circuit = RCParallel(E, 10, 26.53)  
rl_parallel_circuit = RLParallel(E, 10, 37.70)  
rlc_parallel_circuit = RLCParallel(E, 10, 37.70, 26.53)  

In [91]:
print("R Circuit:")
print("Impedance (Z):", r_circuit.format_phasor(r_circuit.impedance))
print("Current (I):", r_circuit.format_phasor(r_circuit.current))
print("Power factor:", round(r_circuit.power_factor, 2), "lagging")
print()

R Circuit:
Impedance (Z): 10.00∠0.00°
Current (I): 2.00∠0.00°
Power factor: 1.0 lagging



In [93]:
print("RC Series Circuit:")
print("Impedance (Z):", rc_series_circuit.format_phasor(rc_series_circuit.impedance))
print("Current (I):", rc_series_circuit.format_phasor(rc_series_circuit.current))
VR, VC = rc_series_circuit.voltage_drops
print("VR:", rc_series_circuit.format_phasor(VR))
print("VC:", rc_series_circuit.format_phasor(VC))
print("Power factor:", round(rc_series_circuit.power_factor, 2), "lagging")
print()

RC Series Circuit:
Impedance (Z): 28.35∠-69.35°
Current (I): 0.71∠69.35°
VR: 7.05∠69.35°
VC: 18.71∠-20.65°
Power factor: 0.35 lagging



In [95]:
print("RL Series Circuit:")
print("Impedance (Z):", rl_series_circuit.format_phasor(rl_series_circuit.impedance))
print("Current (I):", rl_series_circuit.format_phasor(rl_series_circuit.current))
VR, VL = rl_series_circuit.voltage_drops
print("VR:", rl_series_circuit.format_phasor(VR))
print("VL:", rl_series_circuit.format_phasor(VL))
print("Power factor:", round(rl_series_circuit.power_factor, 2), "lagging")
print()

RL Series Circuit:
Impedance (Z): 39.00∠75.14°
Current (I): 0.51∠-75.14°
VR: 5.13∠-75.14°
VL: 19.33∠14.86°
Power factor: 0.26 lagging



In [97]:
print("RLC Series Circuit:")
print("Impedance (Z):", rlc_series_circuit.format_phasor(rlc_series_circuit.impedance))
print("Current (I):", rlc_series_circuit.format_phasor(rlc_series_circuit.current))
VR, VL, VC = rlc_series_circuit.voltage_drops
print("VR:", rlc_series_circuit.format_phasor(VR))
print("VL:", rlc_series_circuit.format_phasor(VL))
print("VC:", rlc_series_circuit.format_phasor(VC))
print("Power factor:", round(rlc_series_circuit.power_factor, 2), "lagging")
print()

RLC Series Circuit:
Impedance (Z): 14.99∠48.16°
Current (I): 1.33∠-48.16°
VR: 13.34∠-48.16°
VL: 50.29∠41.84°
VC: 35.39∠-138.16°
Power factor: 0.67 lagging



In [99]:
print("RC Parallel Circuit:")
print("Impedance (Z):", rc_parallel_circuit.format_phasor(rc_parallel_circuit.impedance))
print("Current (I):", rc_parallel_circuit.format_phasor(rc_parallel_circuit.current))
print("Power factor:", round(rc_parallel_circuit.power_factor, 2), "lagging")
print()

RC Parallel Circuit:
Impedance (Z): 9.36∠-20.65°
Current (I): 2.14∠20.65°
Power factor: 0.94 lagging



In [101]:
print("RL Parallel Circuit:")
print("Impedance (Z):", rl_parallel_circuit.format_phasor(rl_parallel_circuit.impedance))
print("Current (I):", rl_parallel_circuit.format_phasor(rl_parallel_circuit.current))
print("Power factor:", round(rl_parallel_circuit.power_factor, 2), "lagging")
print()

RL Parallel Circuit:
Impedance (Z): 9.67∠14.86°
Current (I): 2.07∠-14.86°
Power factor: 0.97 lagging



In [103]:
print("RLC Parallel Circuit:")
print("Impedance (Z):", rlc_parallel_circuit.format_phasor(rlc_parallel_circuit.impedance))
print("Current (I):", rlc_parallel_circuit.format_phasor(rlc_parallel_circuit.current))
print("Power factor:", round(rlc_parallel_circuit.power_factor, 2), "lagging")


RLC Parallel Circuit:
Impedance (Z): 9.94∠-6.37°
Current (I): 2.01∠6.37°
Power factor: 0.99 lagging


# PREPARED BY SHEHWAR MUHAMMAD